# 2 CNNs en Google Street View

En esta sección, trabajaremos con el dataset **SVHN** (Street View House Numbers), correspondiente a imágenes naturales de dígitos de direcciones obtenidos desde Google Street View. El dataset contiene más de 600.000 imágenes de entrenamiento y 26.032 imágenes de test. Para facilitar la realización de experimentos, el dataset de entrenamiento se divide usualmente en un conjunto pequeño de 73.257 imágenes y un conjunto “*extra*” de 531.131 imágenes. En esta tarea trabajaremos sólo con la versión pequeña. Los valientes pueden veriﬁcar que entrenando sobre el conjunto grande los resultados mejoran signiﬁcativamente.

Los datos pueden ser obtenidos (en formato Matlab) ejecutando los siguientes comandos:

```
wget http://ufldl.stanford.edu/housenumbers/train_32x32.mat 
wget http://ufldl.stanford.edu/housenumbers/extra_32x32.mat
wget http://ufldl.stanford.edu/housenumbers/test_32x32.mat

```

## A. Conocer el DataSet

Cargue los datos de entrenamiento y pruebas (“*train 32x32.mat*” y “*test 32x32.mat*”). Determine el tamaño de las imágenes, el número de clases diferentes y de ejemplos en cada categoría. Finalmente, visualice 5 imágenes de entrenamiento y 5 de test (elegidas aleatoriamente). Comente.

```python
1 import scipy.io as sio 
2 import numpy as np 
3 train_data = sio.loadmat('train_32x32.mat') 
4 test_data = sio.loadmat('test_32x32.mat') 
5 X_train = train_data['X'].T 
6 y_train = train_data['y'] - 1 
7 X_test = test_data['X'].T 
8 y_test = test_data['y'] - 1 
9 X_train = X_train.astype('float32') 
10 X_test = X_test.astype('float32') 
11 n_classes = len(np.unique(y_train)) 
12 print np.unique(y_train)
```

## B. Normalización de datos

Normalice las imágenes, dividiendo las intensidades originales de pixel por 255. Represente adecuadamente la salida deseada de la red de modo de tener un vector de tamaño igual al número de clases.

```python
1 from keras.utils import np_utils 
2 X_train /= 255 
3 X_test /= 255 
4 Y_train = np_utils.to_categorical(y_train, n_classes) 
5 Y_test = np_utils.to_categorical(y_test, n_classes)
```

## C. Definir una red convolucional

Deﬁna una CNN con arquitectura C×P×C×P×F×F. Para la primera capa convolucional utilice 16 ﬁltros de 5×5 y para la segunda 512 ﬁltros de 7×7. Para la capa MLP escondida use 20 neuronas. Esta arquitectura, con algunas diferencias, fue una de las primera CNNs entrenadas sobre SVHN y consiguió una accuracy de 94.28%. Genere un esquema lo más compacto posible que muestre los cambios de forma que experimenta un patrón de entrada a medida que se ejecuta un forward-pass. Entrene la red anterior un máximo de 10 epochs. ¿Logra mejorar o al menos igualar el resultado reportado en la literatura?

```python
1 from keras.models import Sequential 
2 from keras.layers.core import Dense, Dropout, Activation, Flatten 
3 from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D 
4 from keras.optimizers import SGD, Adadelta, Adagrad 
5 model = Sequential() 
6 model.add(Convolution2D(16, 5, 5, border_mode='same', activation='relu', 
7                                 input_shape=(n_channels, n_rows, n_cols))) 
8 model.add(MaxPooling2D(pool_size=(2, 2))) 
9 model.add(Convolution2D(512, 7, 7, border_mode='same', activation='relu')) 
10 model.add(MaxPooling2D(pool_size=(2, 2))) 
11 model.add(Flatten()) 
12 model.add(Dense(20, activation='relu')) 
13 model.add(Dense(n_classes, activation='softmax')) 
14 model.summary() 
15 model.compile(loss='binary_crossentropy', optimizer=adagrad, metrics=['accuracy']) 
16 adagrad = Adagrad(lr=0.01, epsilon=1e-08, decay=0.0) 
17 model.fit(X_train, Y_train, batch_size=1280, nb_epoch=12, verbose=1, \ 
18                                             validation_data=(X_test, Y_test))
```

## D. Modificar tamaño de los filtros

Evalúe el efecto de modiﬁcar el tamaño de los ﬁltros (de convolución y pooling) reportando la sensibilidad del error de pruebas a estos cambios. Presente un gráﬁco o tabla resumen. Por simplicidad entre durante sólo 10 epochs.

## E. Modificar número de filtros

Evalúe el efecto de modiﬁcar el número de ﬁltros para las capas convolucionales tanto en los tiempos de entrenamiento como en el desempeño de la red. Presente un gráﬁco o tabla resumen. Por simplicidad entre durante sólo 10 epochs.

## F. Propuesta de mejora

Proponga una mejora sobre la red deﬁnida en (**c**) que mejore el error de pruebas. Recuerde que debe deﬁnir un subconjunto de validación si necesita elegir entre arquitecturas. 

## G. Visualizacion de pesos y efectos del filtro

Elija una de las redes entrenadas (preferentemente una con buen desempeño) y visualice los pesos correspondientes a los ﬁltros de la primera capa convolucional. Visualice además el efecto del ﬁltro sobre algunas imágenes de entrenamiento. Comente.

## H. Determinar dígitos confusos para la red

Elija una de las redes entrenadas en esta sección y determine los pares de dígitos (por ejemplo “1” con “7”) que la red tiende a confundir. Conjeture el motivo de tal confusión.


## I. Evaluación de convenencia

(Opcional, Bonus +10 en certamen) Evalúe la conveniencia de utilizar todo el dataset (“extra 32x32.mat”) en el entrenamiento de la red.

------------------------

# Desarrollo

## A)

Mediante la utilización de la función *.shape* podemos obtener como últimos dos resultados, las dimensiones de cada elemento. Aún cuando esto sea evidente dado que bajamos un formato *32x32* Además las clases obtenidas en el conjunto **y** pueden ser enumeradas mediante la función *unique()* de **numpy**, lo que genera una lista de todas las clases distintas y luego basta con obtener el largo de la lista para saber la cantidad de estos.

> Tamaño de las imagenes: 32x32

> Número de clases: 10

Luego para ver las imagenes se utiliza la clásica librería de **matplotlib** pero utilizando la funcionalidad *implot()*. Entonces, se toma un conjunto aleatoreo de ambos grupos. **NOTAR** por temas de memoria, el cambio *astype*, se deja de lado por el momento..

```python
TRAIN_EXAMPLES = random.sample(list(X_train), IMG_NUM)
```
<img src="Scripts/Parte 2/5im_entrenamiento.png" style="width: 200px;"/>
```python
TEST_EXAMPLES = random.sample(list(X_test), IMG_NUM)
```
<img src="Scripts/Parte 2/5im_test.png" style="width: 200px;"/>

### Comentarios

El uso del transpose se hace, dado que se intento realizar un reshape antes de ejecutarlo, pero existía una variable 32 que quedaba inmovil lo que seguía dejando la imagen ladeada por lo que se decidió utilizar la propiedad *transpose* para reordenar literalmente el conjunto.

## B)

Se realizan los pasos pedidos, sin embargo **Python 2.7** en **Windows** tiene restringido el uso de memoria para los procesos del mismo, por esto utilizar *float32* fue radicalmente imposible, aún cuando teníamos una CPU con 16GB de RAM. Dada la premisa anterior se tuvo que cambiar a *float16*.

```python
X_train = X_train.astype('float16') 
X_test = X_test.astype('float16') 
```

## C)


```
|-----------------------------|-------------------|----------|
| Layer (type)                | Output Shape      | Param #  |
|-----------------------------|-------------------|----------|
|conv2d_1 (Conv2D)            |(None, 16, 32, 32) |1216      |
|max_pooling2d_1 (MaxPooling2)|(None, 16, 16, 16) |0         |
|conv2d_2 (Conv2D)            |(None, 512, 16, 16)|401920    |
|max_pooling2d_2 (MaxPooling2)|(None, 512, 8, 8)  |0         |
|flatten_1 (Flatten)          |(None, 32768)      |0         |
|dense_1 (Dense)              |(None, 20)         |655380    |
|dense_2 (Dense)              |(None, 10)         |210       |
```

> Total params: 1,058,726

> Trainable params: 1,058,726

> Non-trainable params: 0

